Import stuff

In [29]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
import pandas as pd

Get data

In [30]:
dat = pd.read_csv("train_data.csv", delimiter = ",",encoding= "utf8")
X_str, y = dat["content"], dat["label"]

Transform data

In [31]:
vec =CountVectorizer() 
X = vec.fit_transform(X_str)

X_train, X_test = X[:-1000], X[-1000:]
y_train, y_test = y[:-1000], y[-1000:]

Train KNN

In [6]:
knn = KNeighborsClassifier(n_neighbors= 7)

knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)


print(f"Accuracy: {(knn_pred == y_test).mean():.3}")

Accuracy: 0.846


Train Random Forest

In [32]:
RF = RandomForestClassifier(verbose = 0, max_depth = 50, n_estimators= 10)

RF.fit(X_train,y_train)

# RF_pred = RF.predict(X_test)

# print(f"Accuracy: {(RF_pred == y_test).mean():.3}")

RandomForestClassifier(max_depth=50, n_estimators=10)

In [59]:
accs = []
precs = []
recas = []
f1s =[]
for i in range(3):
    X, y = X_train[(i*(X_train.shape[0]))//10:((i+1)*(X_train.shape[0]))//10], y_train[(i*(X_train.shape[0]))//10:((i+1)*(X_train.shape[0]))//10]
    pred = RF.predict(X)
    acc = (pred == y)
    accs.append(acc.mean())

    TP = (pred & y).mean()
    FP = (pred & ~y).mean()
    TN = (~pred & ~y).mean() 
    FN = (~pred & y).mean() 

    prec =  TP/(TP+FP)  #TP/(TP + FP)

    recall = TP/(TP+FN) # TP/(TP + FN)

    precs.append(prec)
    recas.append(recall)
    f1s.append(TP/(TP+(FP+FN)/2))

In [60]:
def me(ns):
    print(f"mean: {np.mean(ns):.3} err {np.std(ns):.3}")


print("accuracy")
me(accs)
print("precision")
me(precs)
print("recall")
me(recas)
print("f1")
me(f1s)


accuracy
mean: 0.916 err 0.00109
precision
mean: 0.948 err 0.00128
recall
mean: 0.864 err 0.00121
f1
mean: 0.904 err 0.000941


In [17]:
liar = pd.read_csv("liar_dataset/train.tsv", sep='\t' , names=["file","fake","content","tags","speaker","job","state", "political_leaning","n1","n2","n3","n4","n5", "where"])


In [9]:
lX = vec.transform(liar["content"])

liarpred = RF.predict(lX)

In [22]:
trues = (liar["fake"] == "true") | (liar["fake"] == "mostly-true")
falses = (liar["fake"] == "pants-fire") | (liar["fake"] == "false")

liar = liar[trues | falses]
liarpred = liarpred[trues | falses]

tf = (liar["fake"] == "true") | (liar["fake"] == "mostly-true").astype(int)
print(f"Accuracy: {(liarpred  == tf).mean():.3}")


Accuracy: 0.448
